In [1]:
# optional pandas configurations to debug

import pandas as pd

#pd.options.mode.chained_assignment = 'raise'

pd.set_option('display.max_rows', 20)
pd.set_option('display.min_rows', 20)
pd.set_option('display.max_columns', 50)

print('Done')

Done


In [2]:
# Populate core object model
# Note: 
# /Users/ajk/Repositories/helcom_blues/som_tools.py:159: RuntimeWarning: divide by zero encountered in true_divide 
# scaling_factor = np.divide(max_expected_value, max_effectivness)
# warning can be easily fixed by adding 'from np.finfo import eps' and adding eps to max_effectivness

import numpy as np
import pandas as pd

from configuration import input_files
from som_app import process_data
from som_classes import initialize_objects


# Process excel data 
# initialize core object model 
survey_df, object_dicts = process_data()
instance_dicts = initialize_objects(survey_df=survey_df, object_dicts=object_dicts)
measure_instances = instance_dicts['measure'] 


# Rearrange initialized core objects
# and make linkages between core objects
measure_df = pd.DataFrame.from_dict({
    'instance': measure_instances.values()
})

# Note: 
# ID:s are calculated so that they are can be tracked

measure_df['measure id'] = [int(x.id / 10000) * 10000 for x in measure_df['instance']]
measure_df['activity-pressure id'] = [x.activity_pressure.id if x.activity_pressure != None else np.nan for x in measure_df['instance']]
measure_df['activity id'] = [int(x.activity_pressure.id / 10000) * 10000 if x.activity_pressure != None else np.nan for x in measure_df['instance']]
measure_df['pressure id'] = measure_df['activity-pressure id'] - measure_df['activity id']


/Users/ajk/Repositories/helcom_blues/som_tools.py:159: RuntimeWarning: divide by zero encountered in true_divide
  scaling_factor = np.divide(max_expected_value, max_effectivness)


In [3]:
# Going through core objects and setting initial values

for num in measure_df.index:
    m = measure_df.loc[num, 'instance']

    # measures have activity-pressure pairs or states
    if m.states:
        continue

    # setting expected value first on measure instance
    # multiplicating on activity-pressure intance based on measure instance
    # setting pressure reduction value in pressure instance  
    elif m.activity_pressure:
        ap = m.activity_pressure
        p = ap.pressure

        value = m.expected * m.multiplier

        ap.expected = ap.expected * value

        p.pressure_reduction = value

    else:
        raise AttributeError("Measure instance is missing Activity-Pressure pair or State instances.")


In [4]:
# Second object layer is encapsulating core objects in study case instances
# case instances gather measures, countries and basins

from typing import Type
from som_classes import Measure

class Case:

    def __init__(self, id: int) -> None:
        self._id = id
    
    def __repr__(self) -> str:
        return f"Case {self.id}"

    @property
    def id(self) -> int:

        return self._id


class CountryBasin:
    """ Describes areas in Baltic Sea region
    
    Properties:
        name (str): name of human activity
        id (int): identification number of country-basin pair country id * 1000 + basin id
        basin_fraction (float): fraction of countries economic  in basin area
    """

    def __init__(self, id: int, name: str) -> None:
        self._id = id
        self._name = name
        self._basin_fraction = None

        self._measures = None
        self._cases = None
    
    def __repr__(self) -> str:
        return self._name

    @property
    def id(self) -> int:

        return self._id

    @property
    def coutry_id(self) -> int:

        return int(self._id / 1000)
    
    @property
    def basin_id(self) -> int:

        country_id = int(self._id / 1000)
        basin_id = self._id - country_id

        return basin_id

    @property
    def basin_fraction(self) -> float:

        return self._basin_fraction
    
    @basin_fraction.setter
    def basin_fraction(self, value):

        if value <= 0 or value > 1.0:
            raise ValueError("Fraction of basin area have to be between 0 and 1")

        self._basin_fraction = value

    @property
    def measures(self):

        return self._measures

    @measures.setter
    def measures(self, instance: Type[Measure]) -> None:

        if self._measures == None:
            self._measures = []

        self._measures.append(instance)
    
    @property
    def cases(self) -> list[Type[Case]]:

        return self._cases
    
    @cases.setter
    def cases(self, instance: Type[Case]) -> None:

        if self._cases == None:
            self._cases = []
        
        self._cases.append(instance)


In [6]:
# Read in and process country, basin and countries by basin tables
# Instantiate CountryBasin objects

file_name = input_files['general_input']
sheet_name = 'CountBas'

countries_by_basins = pd.read_excel(io=file_name, sheet_name=sheet_name)
index = countries_by_basins[countries_by_basins['country'] == 'All countries'].index
countries_by_basins.drop(index=index, inplace=True)

sheet_name = 'Country list'

countries = pd.read_excel(io=file_name, sheet_name=sheet_name, index_col='ID')
index = countries[(countries.index == 0) | (countries.index == 10)].index
countries.drop(index=index, inplace=True)

sheet_name = 'Basin list'

basins = pd.read_excel(io=file_name, sheet_name=sheet_name, index_col='ID')
index = basins[basins.index == 0].index
basins.drop(index=index, inplace=True)

instances = {}

for country in countries['COUNTRY']:
    country_id = countries[countries['COUNTRY'] == country].index[0]
    
    for basin in basins['Basin']:
       
        basin_fraction = countries_by_basins.loc[(countries_by_basins['country'] == country), basin].values[0]
 
        if basin_fraction <= 0:
            continue

        basin_id = basins[basins['Basin'] == basin].index[0]

        # f"Activity {self.activity.id} and Pressure {self.pressure.id}"

        countrybasin_id = basin_id * 1000 + country_id
        countrybasin_name = f"{country} ({country_id}) and {basin} ({basin_id})"

        cb = CountryBasin(id=countrybasin_id, name=countrybasin_name)
        cb.basin_fraction = basin_fraction

        instances.update({countrybasin_id: cb})

# Rearrange CountryBasin objects to DataFrame
countrybasin_df = pd.DataFrame.from_dict({
    'instance': instances.values()
})

countrybasin_df['country-basin id'] = [x.id for x in countrybasin_df['instance']]
countrybasin_df['basin id'] = [int(x.id / 1000) * 1000 for x in countrybasin_df['instance']]
countrybasin_df['country id'] = countrybasin_df['country-basin id'] - countrybasin_df['basin id']


In [7]:
# Reading in and processing data for cases
# each row represents one case
# in_Activities, in_Pressure and In_State_components 0 == 'all relevant' 
# relevant activities, pressures and state can be found in measure-wise from MT_to_A_to_S
# multiply MT_ID id by 10000 to get right measure_id
# multiply In_Activities ids by 10000 to get right activity_id
# multiply B_ID by 1000

file_name = input_files['general_input']
sheet_name = 'ActMeas'

cases = pd.read_excel(io=file_name, sheet_name=sheet_name)
connection_table = pd.read_excel

cases['In_Activities'] = [list(filter(None, x.split(';'))) if type(x) == str else x for x in cases['In_Activities']]
cases['In_Pressure'] = [list(filter(None, x.split(';'))) if type(x) == str else x for x in cases['In_Pressure']]
cases['In_State_components'] = [list(filter(None, x.split(';'))) if type(x) == str else x for x in cases['In_State_components']]

cases['B_ID'] = [list(filter(None, x.split(';'))) if type(x) == str else x for x in cases['B_ID']]
cases['C_ID'] = [list(filter(None, x.split(';'))) if type(x) == str else x for x in cases['C_ID']]

# Column-by-column

## Sheet ActMeas

Determined by governance issues. All rows are independent.

1. ID - ID of the case
2. MT_ID - Measure type ID
3. In_Activities - Relevant Activities, 0 means all relevant activities for Measure type in MT_to_A_to_P sheet
4. In_Pressure - Relevant Pressures, 0 means all relevant pressures for Measure type in MT_to_A_to_P sheet
5. In_State_components - Relevant States, 0 means all relevant states for Measure type in MT_to_A_to_P sheet
6. multiplier_OL - Measure type multiplier
7. B_ID - Basin ID:s
8. C_ID - Country ID:s

## Sheet ActPres

Determined by environmental issues. Lists the relevant basins for each activity-pressure pair data response.

1. Activity - Activity ID
2. Pressure - Pressure ID
3. Basin - Basin ID
4. Ml# - MostLikely (ActivityPressure.AP_expected)
5. Min# - Minimum end of range 
6. Max# - Maximum end of range


In [14]:
# CountryBasins have their own set of Core objects
# Core objects are trimmed to Cases

# Initialize Case objects
# Fetch correct Measure instances
# Fetch correct CountryBasin instances
# Note: One Measure is applied only once to one CountryBasin
# Note: case_id = cases[ID] * 100'000'000 + measure_id

measure_cases = cases.loc[:, 'MT_ID'] # NOT YET MULTIPLIED
basin_cases = cases.loc[:, 'B_ID']  # NOT YET MULTIPLIED
country_cases = cases.loc[:, 'C_ID']


for num in cases.index:

    df = pd.DataFrame()

    measure_id = measure_cases.loc[num]
    case_id = cases['ID'].loc[num] * 100000000 + measure_id
    
    measures = measure_df.loc[measure_df['measure id'] == measure_id]

    basins = basin_cases.loc[num]
    countries = country_cases.loc[num]

    for basin_id in basins:
        #basin_id = basin_id * 1000

        print(basin_id)

        # All basins is special case
        # in future, handle separately
        if basin_id == 0:
            continue

        for country_id in countries:
            
            # Global is special case
            # in future, handle separately
            if country_id == 10:
                continue
            
            # All Countries is special case
            # in future, handle separately
            if country_id == 0:
                continue

            countrybasin_id = basin_id + country_id

            cb = countrybasin_df.loc[countrybasin_df['country-basin id'] == countrybasin_id, 'instance']
  

1
2
3
4
5
6
7
1
2
3
4
5
6
7
9
10
11
12
13
14
15
16
17
1
2
3
4
5
6
7
9
11
12
13
13
14
15
16
17
4
5
6
7
9
11
9
7
8
9
1
3
6
7
9
10
12
14
15
16
17
1
2
3
4
5
6
7
9
11
12
13
13
14
15
16
17
4
5
6
7
9
11
9
7
8
9
1
3
6
7
9
10
12
14
15
16
17
1
2
3
4
5
6
7
9
11
12
13
13
14
15
16
17
4
5
6
7
9
11
9
7
8
9
1
3
6
7
9
10
12
14
15
16
17
4
5
6
7
0
0
1
2
3
4
5
6
7
9
11
12
13
13
14
15
16
17
4
5
6
7
9
11
9
7
8
9
1
3
6
7
9
10
12
14
15
16
17
1
2
3
4
5
6
7
9
10
11
12
13
14
15
16
17
1
2
3
4
5
6
7
9
11
12
13
13
14
15
16
17
4
5
6
7
9
11
9
7
8
9
1
3
6
7
9
10
12
14
15
16
17
1
2
3
4
5
6
7
9
11
12
13
13
14
15
16
17
4
5
6
7
9
11
9
7
8
9
1
3
6
7
9
10
12
14
15
16
17
1
2
3
4
5
6
7
9
11
12
13
13
14
15
16
17
4
5
6
7
9
11
9
7
8
9
8
9
13
1
3
6
7
9
10
12
14
15
16
17
1
2
3
4
5
6
7
1
2
3
4
5
6
7
1
2
3
4
5
6
7
8
9
13
0
1
3
6
7
9
10
12
14
15
16
17
9
1
3
6
7
9
10
12
14
15
16
17
1
3
6
7
9
10
12
14
15
16
17
12
13
14
15
16
17
1
3
6
7
9
10
12
14
15
16
17
1
2
3
4
5
6
7
5
6
7
9
6
7
5
12
13
14
15
16
17
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15